In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -O data.zip http://extrasensory.ucsd.edu/data/primary_data_files/ExtraSensory.per_uuid_features_labels.zip
!mkdir user_data
!unzip data.zip -d user_data
#!cd user_data && for g in *.gz; do gunzip $g; done

--2024-10-12 16:33:01--  http://extrasensory.ucsd.edu/data/primary_data_files/ExtraSensory.per_uuid_features_labels.zip
Resolving extrasensory.ucsd.edu (extrasensory.ucsd.edu)... 132.239.17.138
Connecting to extrasensory.ucsd.edu (extrasensory.ucsd.edu)|132.239.17.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225374973 (215M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 214.93M  42.6MB/s    in 5.5s    

2024-10-12 16:33:07 (39.3 MB/s) - ‘data.zip’ saved [225374973/225374973]

Archive:  data.zip
  inflating: user_data/00EABED2-271D-49D8-B599-1D4A09240601.features_labels.csv.gz  
  inflating: user_data/098A72A5-E3E5-4F54-A152-BBDA0DF7B694.features_labels.csv.gz  
  inflating: user_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv.gz  
  inflating: user_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv.gz  
  inflating: user_data/0E6184E1-90C0-48EE-B25A-F1ECB7B9714E.features_labels.csv.gz  
 

In [ ]:
#!rm -rf user_data
#!mkdir user_data
#!unzip data.zip -d user_data

# RNN

resource : https://colah.github.io/posts/2015-08-Understanding-LSTMs/

1. Load the extrasensory dataset

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import torch



import gzip
import os
import shutil
import zipfile

In [ ]:

def getinvehicle(row):
    if row['label:ON_A_BUS'] == 1 or row['label:IN_A_CAR'] == 1 or row['label:DRIVE_-_I_M_THE_DRIVER'] == 1 or row['label:DRIVE_-_I_M_A_PASSENGER'] == 1:
        val = 1
    else:
        val = 0
    return val

def dataset_fix():
    with open('sensors.csv') as file:
        featuresToKeep = [line.rstrip() for line in file]

    with open('labels.csv') as file:
        labelsToKeep = [line.rstrip() for line in file]

    #with zipfile.ZipFile('ExtraSensory.per_uuid_features_labels.zip', 'r') as zip_ref:
        #zip_ref.extractall('user_data')

    df_list = []

    for file in os.listdir('user_data'):
        with gzip.open('user_data/'+file, 'rb') as f_in:
            with open(f_in.name.replace('.gz', ''), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    for file in os.listdir('user_data'):
        if file.endswith('.csv'):
            df = pd.read_csv('user_data/'+file)

            df['activityType:in_vehicle'] = df.apply(getinvehicle, axis=1)

            cols = list(df.columns)
            features = list(filter(lambda s: not s.startswith("label"), cols))
            labels = list(filter(lambda s: s.startswith("label"), cols))

            toswap = ["label:FIX_walking", "label:FIX_running", "label:BICYCLING", "label:SLEEPING"]

            features.extend(toswap)
            labels = [x for x in labels if x not in set(toswap)]
            features.extend(labels)

            df = df.reindex(columns=features)
            df.rename(columns={'label:FIX_walking': 'activityType:walking', 'label:FIX_running': 'activityType:running',
                               'label:BICYCLING': 'activityType:on_bicycle', 'label:SLEEPING': 'activityType:sleeping'},
                      inplace=True)


            columnsToRemove = []

            for series_name, series in df.items():
                if series_name not in featuresToKeep and series_name not in labelsToKeep:
                    columnsToRemove.append(series_name)

            df.drop(columns=columnsToRemove, inplace=True)
            #df = df.replace(r'^\s*$', np.nan, regex=True)

            # Save the final result to a new CSV file
            df.to_csv('user_data/'+file, index=False, na_rep='nan')

            #f_in = open('user_data/' + file)
            #f_out = gzip.open('user_data/' + file + 'test.gz', 'wb')
            #f_out.writelines(f_in)
            #f_out.close()
            #f_in.close()

            with open('user_data/' + file, 'rb') as f_in, gzip.open('user_data/' + file + '.gz', 'wb') as f_out:
              f_out.writelines(f_in)

dataset_fix()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class select_features_by_name(BaseEstimator, TransformerMixin):
    '''
    Select feature by given feature names. Compatible with sklearn transformer class.

    Keyword Arguments:
        feature_to_use: [list] -- feature names in list we want to select
        feature_names: [list] -- feature names of all the possible features
    '''

    def __init__(self, feature_to_use, feature_names):

        self.feature_to_use = feature_to_use
        self.feature_names = feature_names

        fi = []
        for i, feature in enumerate(self.feature_names):
            if feature in self.feature_to_use:
                fi.append(i)
        self.fi = fi

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[:, self.fi]

class select_features_by_sensors(BaseEstimator, TransformerMixin):

    '''
    Select feature by given sensor names. Compatible with sklearn transformer class.

    Keyword Arguments:
        sensors_to_use: [list] -- feature names in list we want to select
        feature_names: [list] -- feature names of all the possible features
    '''

    def __init__(self, sensors_to_use, feature_names):
        self.sensors_to_use = sensors_to_use
        self.feature_names = feature_names

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        fi = []
        for i, feature in enumerate(self.feature_names):
            if sensor_name_abbriviation[feature.split(':')[0]] in self.sensors_to_use:
                fi.append(i)

        return X[:, fi]

def select_target_labels(X, y, M, target_labels, label_names, drop_all_zero = True):
    '''
    Given target labels, labels matrix and all the possible label names, return label matrix with only target labels
    '''
    li = []
    for i, label_name in enumerate(label_names):
        if label_name in target_labels:
            li.append(i)

    sample_index = (np.sum(y[:, li], axis = 1) != 0)
    if drop_all_zero:
        return X[sample_index, :], y[sample_index, :][:, li], M[sample_index, :][:, li]
    else:
        return X[:,:], y[:, li], M[:, li]

def split_by_users(X, y, M, test_uuid, user_index):
    '''
    Split the dataset into training and test set given the test users' index in uuid_list
    '''
    X_train, y_train, M_train = [], [], []
    X_test, y_test, M_test = [], [], []

    for i in range(60):
        if i in test_uuid:
            X_test.append(X[user_index[i]:user_index[i+1], :])
            M_test.append(M[user_index[i]:user_index[i+1], :])
            y_test.append(y[user_index[i]:user_index[i+1]])
        else:
            X_train.append(X[user_index[i]:user_index[i+1], :])
            M_train.append(M[user_index[i]:user_index[i+1], :])
            y_train.append(y[user_index[i]:user_index[i+1]])

    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)
    M_train = np.concatenate(M_train)
    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)
    M_test = np.concatenate(M_test)

    return X_train, y_train, M_train, X_test, y_test, M_test


if __name__ == 'main':

    None

In [ ]:

class feature_selector():

	def __init__(self, feature_names):

		self.feature_names = feature_names
		self.selected_features = []

	def forward_sequential_selection(self, X_train, y_train, X_test, y_test, model,
										evaluation = 'BA', report = 'True'):
		'''
		Take splitted dataset and a model, use forward sequential selection to select a
		subset of features that gives the highest score on given test set.

		Keyword Arguments:
			X_train, y_train, X_test, y_test: [narray] -- splitted dataset
			model: [sklearn model] -- model used to fit and predict the dataset
			evaluation: [str] -- specify the type of evaluation (score)
			report: [Boolen] -- whether to report the progress

		'''

		self.n_labels = y_train.shape[-1]
		self.evaluation = evaluation

		selected_features = self.selected_features
		remaining_features = self.feature_names.copy()
		for feature in selected_features:
			remaining_features.remove(feature)
		score = 0

		for i in range(len(remaining_features)):
			flag = 0

			# Try all the features that has not been selected
			for j, feature in enumerate(remaining_features):

	        	# Select features
				temp_features = selected_features + [feature]
				feature_selector = select_features_by_name(temp_features, self.feature_names)
				temp_X_train = feature_selector.fit_transform(X_train)
				temp_X_test = feature_selector.transform(X_test)

				# Fit the model
				model.fit(temp_X_train, y_train)

				# predict and calcuate score
				if evaluation == 'model_default':
					temp_score = model.score(temp_X_test, y_test)
				elif evaluation == 'BA':
					_, _, _, temp_score = evaluate_model(model, temp_X_test, y_test, report = False)
				else:
					raise NameError('Evaluation does not exist!')

				# If the score increases, update
				if temp_score > score:

					score = temp_score
					added_feature = feature
					flag = 1
				print('Try feature: %s\t[%d/%d]\t score: %f' %(feature, j, len(remaining_features), temp_score))

			if flag == 0:
				break

	        # Update current selected features and remaining feature to be selected
			selected_features = selected_features + [added_feature]
			remaining_features.remove(added_feature)

			if report:
				print('\nAdd %s\t%d features selected\tscore %f' %(added_feature, len(selected_features), score))

		self.selected_features = selected_features

		return selected_features, score

if __name__ == 'main':

	None

In [ ]:
import gzip
import io as StringIO
import os

import numpy as np
import pandas as pd

def parse_header_of_csv(csv_str):
    # Isolate the headline columns:
    headline = csv_str[:csv_str.index('\n')]
    columns = headline.split(',')

    # The first column should be timestamp:
    assert columns[0] == 'timestamp'
    # The last column should be label_source:
    #assert columns[-1] == 'label_source'

    # Search for the column of the first label:
    for (ci,col) in enumerate(columns):
        if col.startswith('label:'):
            first_label_ind = ci
            break
        pass

    # Feature columns come after timestamp and before the labels:
    feature_names = columns[1:first_label_ind]
    # Then come the labels, till the one-before-last column:
    label_names = columns[first_label_ind:-1]
    for (li,label) in enumerate(label_names):
        # In the CSV the label names appear with prefix 'label:', but we don't need it after reading the data:
        assert label.startswith('label:')
        label_names[li] = label.replace('label:','')
        pass

    return (feature_names,label_names)

def parse_body_of_csv(csv_str,n_features):
    # Read the entire CSV body into a single numeric matrix:
    full_table = np.loadtxt(StringIO.StringIO(csv_str),delimiter=',',skiprows=1)
    # Timestamp is the primary key for the records (examples):
    timestamps = full_table[:,0].astype(int)

    # Read the sensor features:
    X = full_table[:,1:(n_features+1)]

    # Read the binary label values, and the 'missing label' indicators:
    trinary_labels_mat = full_table[:,(n_features+1):-1] # This should have values of either 0., 1. or NaN
    M = np.isnan(trinary_labels_mat) # M is the missing label matrix
    Y = np.where(M,0,trinary_labels_mat) > 0. # Y is the label matrix

    return (X,Y,M,timestamps)

'''
Read the data (precomputed sensor-features and labels) for a user.
This function assumes the user's data file is present.
'''
def read_user_data(uuid):
    user_data_file = 'user_data/%s.features_labels.csv' % uuid

    # Read the entire csv file of the user:
    #with gzip.open(user_data_file,'rb') as fid:
        #csv_str = fid.read().decode()
        #pass

    #user_data_file = 'user_data/%s.features_labels.csv' % uuid
    #with open(user_data_file,'rb') as file:
      #csv_str = file.read()
      #pass

    with open(user_data_file, 'r') as csvfile:
      separator = ''
      csv_str = separator.join(csvfile.readlines())

    (feature_names,label_names) = parse_header_of_csv(csv_str)
    n_features = len(feature_names)
    (X,Y,M,timestamps) = parse_body_of_csv(csv_str,n_features)

    return (X,Y,M,timestamps,feature_names,label_names)


'''
Load the data of all the users
'''

UUID_LIST = []

for file in os.listdir('user_data'):
    if file.split('.')[-1] == 'gz':
        UUID_LIST.append(file.split('.')[0])
print('Found %d users data.' %(len(UUID_LIST)))

def load_all_data(uuid_list = UUID_LIST):
    '''
    Load data from all the sixty users

    Returns:
        X: [narray] -- feature matrix in shape of [n_smaples, n_features]
        y: [narray] -- label matrix in shape of [n_smaples, n_labels]
        M: [narray] -- missing label matrix, element is false means that the label is missing
        user_index: [list] -- indicate the starting index of each user data
        feature_names: [list] -- feature names of all the possible features
        label_names: [list] -- label names of all the possible labels
    '''

    X, y, M = [], [], []
    user_index = [0]

    for i, uuid in enumerate(uuid_list):

        X_i,y_i,M_i,timestamps,feature_names,label_names = read_user_data(uuid)
        user_index.append(user_index[i]+X_i.shape[0])

        X.append(X_i)
        y.append(y_i)
        M.append(M_i)

    X = np.concatenate(X)
    y = np.concatenate(y)
    M = np.concatenate(M)

    return X, y, M, user_index, feature_names, label_names

#if __name__ == 'main':

    # load the data of the first user in the list
    #X,Y,M,timestamps,feature_names,label_names = read_user_data(UUID_LIST[0])

    # Load the data of all the users
    #X, y, M, user_index, feature_names, label_names = load_all_data()


Found 60 users data.


In [ ]:
def score_function(y_test, y_pred, score = 'BA', W_test = None):

    mcm = []
    for i in range(y_test.shape[1]):
        if W_test is not None:
            cm = confusion_matrix(y_test[:,i].T, y_pred[:,i].T, sample_weight = W_test[:,i].T)
        else:
            cm = confusion_matrix(y_test[:,i].T, y_pred[:,i].T)
        cm = np.expand_dims(cm, axis = 0)
        mcm.append(cm)

    mcm = np.concatenate(mcm, axis = 0)
    tn = mcm[:, 0, 0]
    tp = mcm[:, 1, 1]
    fn = mcm[:, 1, 0]
    fp = mcm[:, 0, 1]

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    BA = (sensitivity + specificity)/2
    accuracy = (tn + tp)/(tn + tp + fn + fp)

    sensitivity = np.sum(sensitivity)/sensitivity.shape[0]
    specificity = np.sum(specificity)/specificity.shape[0]
    BA = np.sum(BA)/BA.shape[0]
    accuracy = np.sum(accuracy)/accuracy.shape[0]

    if score == 'BA':
        return BA
    else:
        raise Exception('score not valid!')

In [ ]:
# Load all the data from Extrasenory dataset
X, y, M, user_index, feature_names, label_names = load_all_data()

In [ ]:
import torch

# Only select body state label
#target_label = ['LYING_DOWN', 'SITTING', 'FIX_walking', 'FIX_running', 'BICYCLING', 'OR_standing']
target_label = ["SITTING"]
# Use the last 5 user's data as test set
#test_uuid = list(range(56, 61))

# Fill the Nan with mean value and normalize all the data
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('std_scaler', StandardScaler())
])

# Transform
# 1. select target labels
# 2. tansform feature matrix fill None with mean and do the normalization
# 3. Split train, validation and test set by ratio of 6:2:2
#X_new, y_new, M_new = select_target_labels(X175, y, M, target_label, label_names, drop_all_zero = False)
#X_new, y_new, M_new = select_target_labels(X, y, M, target_label, label_names, drop_all_zero = False)
#X_new = pipeline.fit_transform(X_new, y_new)
#type(X_new)
#X_train, y_train, M_train, X_val, y_val, M_val, X_test, y_test, M_test = random_split(X_new, y_new, M_new, test_size = 0.2, val_size = 0.2, random_seed = 42)
#X_train, y_train, M_train, X_val, y_val, M_val, X_test, y_test, M_test = random_split(X_new, y_new, M_new, test_size = 0.2, val_size = 0.2, random_seed = 42)

In [ ]:
from sklearn.model_selection import train_test_split

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('std_scaler', StandardScaler())
])

X_new = pipeline.fit_transform(X, y)


X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size= 0.2)


# LSTM model

In [ ]:
# [samples, time_steps, features]
X_train_1 = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_1 = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM, Input

hidden_size = 1
time_step = 1


model_2 = Sequential()
model_2.add(Input(batch_shape = (hidden_size, time_step, X_train_1.shape[2])))
model_2.add(LSTM(1, activation = "sigmoid"))
model_2.add(Dropout(0.2))
model_2.add(Dense(14, activation = "sigmoid"))
model_2.compile(loss = "binary_crossentropy",
              optimizer = 'sgd',
              metrics = ['accuracy'])

# fit network
model_lstm = model_2.fit(X_train_1, y_train,
                         epochs = 10,
                         batch_size = 1,
                         validation_data = (X_test_1, y_test))
                         #sample_weight = y_train_sw1D)


try:
  model_2.save("/drive/MyDrive/Tesi/model_2.keras")
except:
  print("save 1 error")


try:
  model_2.save("/content/drive/MyDrive/Tesi/model_2.keras")
except:
  print("save 2 error")

Epoch 1/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 633s 2ms/step - accuracy: 0.0952 - loss: 0.1306 - val_accuracy: 0.0773 - val_loss: 0.0738
Epoch 2/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 685s 2ms/step - accuracy: 0.0779 - loss: 0.0742 - val_accuracy: 0.0773 - val_loss: 0.0726
Epoch 3/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 746s 2ms/step - accuracy: 0.0774 - loss: 0.0734 - val_accuracy: 0.0773 - val_loss: 0.0722
Epoch 4/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 683s 2ms/step - accuracy: 0.0783 - loss: 0.0736 - val_accuracy: 0.0773 - val_loss: 0.0719
Epoch 5/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 626s 2ms/step - accuracy: 0.0781 - loss: 0.0734 - val_accuracy: 0.0773 - val_loss: 0.0718
Epoch 6/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 672s 2ms/step - accuracy: 0.0776 - loss: 0.0725 - val_accuracy: 0.0773 - val_loss: 0.0716
Epoch 7/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 674s 2ms/step - accuracy: 0.0782 - loss: 0.0728 - val_accuracy: 0.0773 - val_loss: 0.0715
Epoch 8/10
301876/301876 ━━━━━━━━━━━━━━━━━━━━ 68

In [ ]:
#test predict modello

In [ ]:

try:
  model_2.save("/model.keras")
except:
  print("save 2 error")

In [ ]:
print(model_lstm.history)

In [ ]:
# Plot training & validation accuracy
#plt.plot(score_rnn)
list_lstm = model_lstm.history['val_accuracy'] + [0.6942]
#list_rnn = model.history["acc"]

fig = plt.figure(figsize = (16, 10))
#plt.plot(model.history['acc'])
plt.plot(list[0])
plt.ylabel('Balanced Accuracy')
plt.xlabel('Epoch')
fig.show()

In [ ]:
#plt.plot(model_rnn.history['val_acc'], "RNN")
plt.plot(model_lstm.history['val_accuracy'])
plt.title("Accuarcy Comparison")
plt.ylabel('Balanced Accuracy')
plt.xlabel('Epoch')
plt.legend(['RNN', 'RNN with LSTM'], loc = 'upper left')
plt.show()